In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf 

#Import our input dataset
attrition_df = pd.read_csv('HR-Employee-Attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [2]:
# Generate categorical variables list 
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()

In [3]:
#Before encoding variables we need to know that none of variables have more than 10 unique values or require bucketing
# Check the number of unique values in each column 
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [4]:
# Create OneHotEncoder Instance 
enc = OneHotEncoder(sparse=False)

#Fit and transform the OneHotEncoder using the categorical variable list 
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame 
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [5]:
# Merge one-hot encoded features and drop the originals 
attrition_df = attrition_df.merge(encode_df, left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat, 1)
attrition_df

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,884,23,2,1,2061,3,41,4,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1466,39,613,6,1,1,2062,4,42,2,3,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1467,27,155,4,3,1,2064,2,87,4,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
1468,49,1023,2,3,1,2065,4,63,2,2,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [6]:
# Split our preprocessed data into our features and target arrays 
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes", "Attrition_No"], 1).values

# Split the preprocessed data into a training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [7]:
# Create a StandardScaler Instance 
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
#Define the model - deep neural net 
number_input_features = len(X_train[0])
hidden_nodes_layers1 = 24
hidden_nodes_layers2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer 
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layers1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers2, activation="relu"))

#Output later
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                1344      
_________________________________________________________________
dense_1 (Dense)              (None, 12)                300       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 13        
Total params: 1,657
Trainable params: 1,657
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [10]:
# Compile the model 
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
# Train the model 
fit_model = nn.fit(X_train, y_train, epochs=100)

Epoch 1/100
35/35 [==============================] - 0s 733us/step - loss: 545.6478 - accuracy: 0.4910
Epoch 2/100
35/35 [==============================] - 0s 638us/step - loss: 134.1851 - accuracy: 0.7623
Epoch 3/100
35/35 [==============================] - 0s 645us/step - loss: 62.9516 - accuracy: 0.7393
Epoch 4/100
35/35 [==============================] - 0s 645us/step - loss: 35.7550 - accuracy: 0.6770
Epoch 5/100
35/35 [==============================] - 0s 675us/step - loss: 17.7179 - accuracy: 0.7516
Epoch 6/100
35/35 [==============================] - 0s 645us/step - loss: 11.3061 - accuracy: 0.7805
Epoch 7/100
35/35 [==============================] - 0s 675us/step - loss: 7.4628 - accuracy: 0.7594
Epoch 8/100
35/35 [==============================] - 0s 704us/step - loss: 9.4906 - accuracy: 0.7340
Epoch 9/100
35/35 [==============================] - 0s 675us/step - loss: 8.4466 - accuracy: 0.7498
Epoch 10/100
35/35 [==============================] - 0s 704us/step - loss: 10.4682

35/35 [==============================] - 0s 616us/step - loss: 2.5633 - accuracy: 0.7717
Epoch 81/100
35/35 [==============================] - 0s 616us/step - loss: 5.0100 - accuracy: 0.7360
Epoch 82/100
35/35 [==============================] - 0s 616us/step - loss: 4.2714 - accuracy: 0.7076
Epoch 83/100
35/35 [==============================] - 0s 645us/step - loss: 6.5657 - accuracy: 0.7481
Epoch 84/100
35/35 [==============================] - 0s 616us/step - loss: 10.3603 - accuracy: 0.6762
Epoch 85/100
35/35 [==============================] - 0s 645us/step - loss: 3.1341 - accuracy: 0.7552
Epoch 86/100
35/35 [==============================] - 0s 616us/step - loss: 5.1946 - accuracy: 0.7433
Epoch 87/100
35/35 [==============================] - 0s 616us/step - loss: 1.7371 - accuracy: 0.7941
Epoch 88/100
35/35 [==============================] - 0s 587us/step - loss: 1.2771 - accuracy: 0.7920
Epoch 89/100
35/35 [==============================] - 0s 587us/step - loss: 5.0235 - accuracy:

In [12]:
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 3.9801 - accuracy: 0.8614
Loss: 3.9801228046417236, Accuracy: 0.8614130616188049


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
35/35 [==============================] - 0s 675us/step - loss: 0.5517 - accuracy: 0.6896
Epoch 2/100
35/35 [==============================] - 0s 645us/step - loss: 0.4518 - accuracy: 0.7914
Epoch 3/100
 1/35 [..............................] - ETA: 0s - loss: 0.2470 - accuracy: 0.9062
Epoch 00003: saving model to checkpoints\weights.03.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.4200 - accuracy: 0.8265
Epoch 4/100
35/35 [==============================] - 0s 645us/step - loss: 0.3509 - accuracy: 0.8522
Epoch 5/100
35/35 [==============================] - 0s 645us/step - loss: 0.3542 - accuracy: 0.8646
Epoch 6/100
 1/35 [..............................] - ETA: 0s - loss: 0.5776 - accuracy: 0.8125
Epoch 00006: saving model to checkpoints\weights.06.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.3562 - accuracy: 0.8504
Epoch 7/100
35/35 [==============================] - 0s 704us/step - loss: 0.3536 - accuracy: 0.8600
Epoch 8/100
35/35

Epoch 56/100
35/35 [==============================] - 0s 616us/step - loss: 0.1018 - accuracy: 0.9746
Epoch 57/100
35/35 [==============================] - 0s 675us/step - loss: 0.0945 - accuracy: 0.9786
Epoch 58/100
 1/35 [..............................] - ETA: 0s - loss: 0.0732 - accuracy: 1.0000
Epoch 00058: saving model to checkpoints\weights.58.hdf5
35/35 [==============================] - 0s 939us/step - loss: 0.0933 - accuracy: 0.9800
Epoch 59/100
35/35 [==============================] - 0s 645us/step - loss: 0.0952 - accuracy: 0.9760
Epoch 60/100
 1/35 [..............................] - ETA: 0s - loss: 0.0732 - accuracy: 0.9688
Epoch 00060: saving model to checkpoints\weights.60.hdf5
35/35 [==============================] - 0s 968us/step - loss: 0.0886 - accuracy: 0.9778
Epoch 61/100
35/35 [==============================] - 0s 616us/step - loss: 0.0911 - accuracy: 0.9726
Epoch 62/100
35/35 [==============================] - 0s 645us/step - loss: 0.0838 - accuracy: 0.9777
Epoch 

In [14]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.7203 - accuracy: 0.8641
Loss: 0.7203218936920166, Accuracy: 0.864130437374115


In [15]:
# Export our model to HDF5 File
nn_new.save("trained_attrition.h5")

In [16]:
# import the model into the notebook
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
#Imported model able to produce exact same results

12/12 - 0s - loss: 0.7203 - accuracy: 0.8641
Loss: 0.7203218936920166, Accuracy: 0.864130437374115
